# Introduction

## AutoPandas
<img src="./imgs/autopandas-logo.png" alt="AutoPandas Logo" style="width: 10%; float: left; padding: 10px"/>


[AutoPandas](https://autopandas.io) is a input-output example based synthesis engine for the [Pandas](https://pandas.pydata.org) data-analytics library in Python. Users provide input-output pairs (dataframes) specifying their intent, and the AutoPandas engine searches for programs using the Pandas library that transform the input to the output.

## Atlas

[Atlas](https://github.com/rbavishi/atlas) is the framework instantiating AutoPandas. It generalizes the key ideas behind its synthesis engine making it possible to apply them to build engines for entirely new domains such as other APIs like Numpy or Tensorflow, or domain-specific languages (DSLs) for say string-manipulation.

## What are we going to learn?

Pandas is a huge library, and building a synthesizer for the entirety of Pandas is out-of-scope for the tutorial.
Instead we will pick a single function in Pandas, namely `pivot`, and try to build an efficient synthesizer for it. That is, given input and output dataframes, our synthesizer will determine the right arguments to the `pivot` function.

We will be using the the abstractions provided by Atlas and in doing so, we hope to motivate why we feel these abstractions are useful for instantiating engines for other domains.

----------------------------------------------

## 1. The Synthesis Task

The user provides input dataframe(s) as well as an output dataframe. Our system should be able to produce a call or a sequence of calls using `pivot`, `groupby` and various aggregation functions that transform the input to the desired output. Here's an example - 

In [1]:
import pandas as pd
from IPython.display import display

inp_df = pd.DataFrame({
  'foo': ['one', 'one', 'one', 'two', 'two', 'two'],
  'bar': ['A', 'B', 'C', 'A', 'B', 'C'],
  'baz': [10, 20, 30, 40, 50, 60],
})

out_df = pd.DataFrame({
    'A': {'one': 10, 'two': 40},
    'B': {'one': 20, 'two': 50},
    'C': {'one': 30, 'two': 60}
})

print("Input DataFrame")
display(inp_df)

print("Output DataFrame")
display(out_df)

Input DataFrame


,foo,bar,baz
0,one,A,10
1,one,B,20
2,one,C,30
3,two,A,40
4,two,B,50
5,two,C,60


Output DataFrame


,A,B,C
one,10,20,30
two,40,50,60


We need to write a method `synthesize_pivot` that when run on the input-output example as follows -

```python
synthesize_pivot(inp_df, out_df)
```

returns the following program or an equivalent program

```python
inp_df.pivot(index="foo", columns="bar", values="baz")
```

## 2. A Brute-Force Synthesizer for Pivot

Before we start building our synthesizer, we need to know the space of programs we are going to search over.

### (a) What are the Possible Programs?

Here's the documentation for pivot -

## Documentation for **Pivot**
<img src="./imgs/pandas-pivot-doc.png" alt="Pivot Documentation" style="width: 80%;"/>

There is a lot of information here! But here are the key points to help us get started -

1. The `index` must be one of the columns of the input table or the value `None`.
2. The `columns` argument must only be one of the columns and NOT `None`.
3. The `values` argument must also be one of the columns or `None`. It can also be a list of columns.

The set of possible arguments to `pivot` is therefore the cross-product of all the possible values for each of the arguments (`index`, `columns` and `pivot`) individually. How can we express this succintly? Now comes in the concept of a `generator` in Atlas/AutoPandas.

### (b) Introduction to Generators

In [2]:
from atlas import generator

@generator
def gen_column_tuples(df: pd.DataFrame):
    """
    Produces all possible 2-tuples of column names
    """
    col1 = Select(df.columns)  # Select one of the columns
    col2 = Select(df.columns)  # Select one of the columns
    
    return (col1, col2)  # Construct the tuple and return

inp_df = pd.DataFrame({
  'foo': ['one', 'one', 'one', 'two', 'two', 'two'],
  'bar': ['A', 'B', 'C', 'A', 'B', 'C'],
  'baz': [10, 20, 30, 40, 50, 60],
})

# Print all possible tuples that can be returned by gen_column_tuples
# In this case all 3x3 = 9 tuples will be returned
for result in gen_column_tuples.generate(inp_df):
    print("Tuple :", result)

Tuple : ('foo', 'foo')
Tuple : ('foo', 'bar')
Tuple : ('foo', 'baz')
Tuple : ('bar', 'foo')
Tuple : ('bar', 'bar')
Tuple : ('bar', 'baz')
Tuple : ('baz', 'foo')
Tuple : ('baz', 'bar')
Tuple : ('baz', 'baz')


The `Select` operator is provided by Atlas. Given a list of values, it returns *one* of the values as the result. The `@generator` transforms any function into an Atlas Generator and overloads any calls to operators like `Select`. One can call `generate(*args, **kwargs)` on such a generator to get an iterator that returns the result of all possible executions of the generator (as governed by the calls to `Select`).

By default, the iterator has **depth-first behavior**. That is, later calls to `Select` explore all possibilities before the previous `Select` calls.

**NOTE** : An Atlas `generator` is different from the Python Generator. Everything that can be expressed using a python generator can be expressed as an Atlas generator. The power of Atlas generators is in combining these generators with probabilistic models as we will see later.

### (c) A Generator for Pivot Arguments

At this point, you should have a fair idea of how to encode the constraints described in **2.(a)** using a generator. Let us try to write a generator for the arguments to the `pivot` function.

In [3]:
@generator
def gen_pivot_args(input_df: pd.DataFrame):
    # Select one of columns
    arg_columns = Select(list(input_df.columns))
    
    # Select one of columns or None
    arg_index = Select([None] + list(input_df.columns))
    
    # Whether to use a column or a list of columns
    if Select([True, False]):
        # Select one of the columns or None
        arg_values = Select([None] + list(input_df.columns))
    else:
        # Pick a Permutation/Ordered-Subset of the list of columns
        arg_values = list(OrderedSubset(list(input_df.columns)))
    
    return {'index': arg_index, 'columns': arg_columns, 'values': arg_values}

We are also ready to write the `synthesize_pivot` function.

In [4]:
def check_equal(df1, df2):
    try:
        pd.testing.assert_frame_equal(df1, df2, check_names=False)
        return True
    except:
        return False
    
def synthesize_pivot(inp, out):
    for args in gen_pivot_args.generate(inp):
        try:
            # If there are exceptions while running pivot or checking, skip
            result = inp.pivot(**args)
            if check_equal(result, out):
                print("Found Solution!", 
                      f"inp.pivot(index='{args['index']}', columns='{args['columns']}', values='{args['values']}')")
        except Exception as e:
            continue

Let's try it out on the example we had in **Section 1**!

In [5]:
inp_df = pd.DataFrame({
  'foo': ['one', 'one', 'one', 'two', 'two', 'two'],
  'bar': ['A', 'B', 'C', 'A', 'B', 'C'],
  'baz': [10, 20, 30, 40, 50, 60],
})

out_df = pd.DataFrame({
    'A': {'one': 10, 'two': 40},
    'B': {'one': 20, 'two': 50},
    'C': {'one': 30, 'two': 60}
})

synthesize_pivot(inp_df, out_df)

Found Solution! inp.pivot(index='foo', columns='bar', values='baz')


## 3. Performance Analysis

Let us log some statistics about our `synthesize_pivot` routine. In particular, let's track the number of programs explored, time taken and number of exceptions raised (while executing the program).

In [6]:
import time

def synthesize_pivot(inp, out):
    start_time = time.time()
    num_explored = 0
    num_errors = 0
    for args in gen_pivot_args.generate(inp):
        num_explored += 1
        
        try:
            # If there are exceptions while running pivot or checking, skip
            result = inp.pivot(**args)
            if check_equal(result, out):
                print("Found Solution!", 
                      f"inp.pivot(index='{args['index']}', columns='{args['columns']}', values='{args['values']}')")
                print(f"Time to solution: {time.time() - start_time: .3f} seconds")
                print(f"Number of Programs Explored Till Now: {num_explored}")

        except Exception as e:
            num_errors += 1
            continue
            
    print(f"Total Number of Programs Explored: {num_explored}")
    print(f"Total Number of Programs Crashed: {num_errors}")

In [7]:
synthesize_pivot(inp_df, out_df)

Found Solution! inp.pivot(index='foo', columns='bar', values='baz')
Time to solution:  0.196 seconds
Number of Programs Explored Till Now: 99
Total Number of Programs Explored: 228
Total Number of Programs Crashed: 57


The solution was produced fairly quickly. But `57` of the `228` argument combinations returned by our generator `gen_pivot` errored out! This means our synthesizer is wasting time evaluating programs that do not even produce an output. Can we could optimize our synthesizer to avoid wasting time on these programs?

### (a) Incorporating Domain-Specific Constraints in Generator

Can we exploit our knowledge of the pivot function to avoid enumerating these programs? Turns out, a meaningful `pivot` program satisfies the following -

1. `columns` and `index` argument cannot be equal
2. The column(s) in `values` must be different from both `columns` and `index`.

It is very easy to incorporate these constraints in the generator as it is regular Python code.

In [8]:
@generator
def gen_pivot_args(input_df: pd.DataFrame):
    # Select one of columns
    arg_columns = Select(list(input_df.columns))
    
    # Select one of columns or None
    arg_index = Select([None] + [i for i in input_df.columns if i != arg_columns]) # CONSTRAINT-1
    
    # Whether to use a column or a list of columns
    if Select([True, False]):
        # Select one of the columns or None
        arg_values = Select([None] + [i for i in input_df.columns if i not in {arg_columns, arg_index}]) # CONSTRAINT-2
    else:
        # Pick a Permutation/Ordered-Subset of the list of columns
        arg_values = list(OrderedSubset([i for i in input_df.columns if i not in {arg_columns, arg_index}])) # CONSTRAINT-2
    
    return {'index': arg_index, 'columns': arg_columns, 'values': arg_values}

In [9]:
synthesize_pivot(inp_df, out_df)

Found Solution! inp.pivot(index='foo', columns='bar', values='baz')
Time to solution:  0.078 seconds
Number of Programs Explored Till Now: 22
Total Number of Programs Explored: 39
Total Number of Programs Crashed: 0


Much better! We are getting to the solution much faster as we are not wasting time exploring bad programs.

### (b) Handling Big DataFrames

We are still doing a brute-force search. Performance would suffer if the space of possible programs itself is pretty large. In the case of `pivot`, what happens if our dataframe has a large number of columns?

In [10]:
inp_df = pd.DataFrame({
    'Date': {0: '2018-02-18', 1: '2018-02-18', 2: '2018-02-24', 3: '2018-02-24'},
    'Location': {0: 'Terrace', 1: 'Pox', 2: 'Gate 320', 3: 'Pox'},
    'Balance': {0: 9971.66, 1: 9726.03, 2: 9604.14, 3: 9356.04},
    'Added By': {0: 'Theresa', 1: 'Margaret', 2: 'Helena', 3:'Katherine'},
    'Expense': {0: 98.34, 1: 245.63, 2: 121.89, 3: 248.0},
    'Category': {0: 'Social', 1: 'Lunch', 2: 'Social', 3: 'Lunch'}})

out_df = pd.DataFrame({'Lunch': {'2018-02-18': 245.63, '2018-02-24': 248.0},
 'Social': {'2018-02-18': 98.34, '2018-02-24': 121.89}})

print("Input DataFrame")
display(inp_df)
print("Output DataFrame")
display(out_df)

synthesize_pivot(inp_df, out_df)

Input DataFrame


,Date,Location,Balance,Added By,Expense,Category
0,2018-02-18,Terrace,9971.66,Theresa,98.34,Social
1,2018-02-18,Pox,9726.03,Margaret,245.63,Lunch
2,2018-02-24,Gate 320,9604.14,Helena,121.89,Social
3,2018-02-24,Pox,9356.04,Katherine,248.00,Lunch


Output DataFrame


,Lunch,Social
2018-02-18,245.63,98.34
2018-02-24,248.00,121.89


Found Solution! inp.pivot(index='Date', columns='Category', values='Expense')
Time to solution:  7.823 seconds
Number of Programs Explored Till Now: 3716
Total Number of Programs Explored: 4056
Total Number of Programs Crashed: 138


Whoops! That is quite a bit slow. If we wanted to build a synthesis service where people can get their input-output queries answered in real-time, this approach would definitely not work.

## 5. Training a Generator

The order of argument combinations returned by `gen_pivot` is governed by the order of values returned by the individual operators (four `Select`s and one `OrderedSubset`). Therefore, to speed up our synthesizer, the operators need to return the correct values *first*. 

#### Can we train the individual operators to return values **smartly** i.e. adjust the order in which they return values based on the input-output example?
##### For the purpose of the tutorial, let's focus on the first call to `Select` i.e. the operator that decides the value of the `columns` argument.

### (a) Adding Input-Output Example as the Context

In order to train the operator, we first need to provide access to the input-output example. Currently only `inp_df` is passed as an argument to the generator `gen_pivot`.

In [11]:
@generator
def gen_pivot_args(input_df: pd.DataFrame, output_df: pd.DataFrame):
    # Select one of columns
    arg_columns = Select(list(input_df.columns), context=(input_df, output_df),  # ADDING CONTEXT HERE
                         uid="select_columns") # Adding uid to aid identification while defining model
    
    # Select one of columns or None
    arg_index = Select([None] + [i for i in input_df.columns if i != arg_columns])
    
    # Whether to use a column or a list of columns
    if Select([True, False]):
        # Select one of the columns or None
        arg_values = Select([None] + [i for i in input_df.columns if i not in {arg_columns, arg_index}])
    else:
        # Pick a Permutation/Ordered-Subset of the list of columns
        arg_values = list(OrderedSubset([i for i in input_df.columns if i not in {arg_columns, arg_index}]))
    
    return {'index': arg_index, 'columns': arg_columns, 'values': arg_values}

In [12]:
def synthesize_pivot(inp, out):
    start_time = time.time()
    num_explored = 0
    num_errors = 0
    for args in gen_pivot_args.generate(inp, out):  # CHANGED : PASSING out AS AN ARGUMENT
        num_explored += 1
        
        try:
            # If there are exceptions while running pivot or checking, skip
            result = inp.pivot(**args)
            if check_equal(result, out):
                print("Found Solution!", 
                      f"inp.pivot(index='{args['index']}', columns='{args['columns']}', values='{args['values']}')")
                print(f"Time to solution: {time.time() - start_time: .3f} seconds")
                print(f"Number of Programs Explored Till Now: {num_explored}")

        except Exception as e:
            num_errors += 1
            continue
            
    print(f"Total Number of Programs Explored: {num_explored}")
    print(f"Total Number of Programs Crashed: {num_errors}")

### (b) Defining the Model

We are going to use a Graph-Neural-Network model for our Select operator. Covering this model is out-of-scope for this tutorial, but here are the key insights behind using this model.

1. The transformation represented by an input-output example consisting of dataframes is a function of the relationships between the values in the input and the values in the output, rather the concrete values themselves.  For example, the concrete column names `Category`, `Expense` etc. are irrelevant. It is their position in the output dataframe that really captures the transformation.

2. We can represent dataframes as a graph where each column, cell and index value is represented as a node, and edges represent relationships amongst these nodes. The nodes are labeled with the type of the value (int, str etc.) rather than the value itself. For example, each column node will have a `COLUMN` edge to all the cell nodes in the corresponding column. We will also have an `EQUALITY` edge between between each pair of nodes that have the same concrete value.

Here is a graphical representation of the graph encoding for the given input and output dataframe along with the domain for the `Select` operator we're trying to learn a model for.

<img src="./imgs/input-output-example.png" alt="Pivot Documentation" style="width: 80%;"/>
<img src="./imgs/graph-encoding-example.png" alt="Pivot Documentation" style="width: 80%;"/>

Let's instantiate the model for the generator. For more details, refer to the `models.py` file in the root directory.

In [13]:
from atlas.models.imitation import IndependentOperatorsModel
from atlas.operators import operator
from models import PivotSelectModel

class PivotGeneratorModel(IndependentOperatorsModel):
    @operator(name='Select', uid="select_columns")
    def SelectColumns(*args, **kwargs):
        config = {
            'learning_rate': 0.01,
            'node_dimension': 50,
            'classifier_hidden_dims': [50],
            'batch_size': 30000,  # number of nodes in a batch
            'layer_timesteps': [1, 1, 1]
        }

        return PivotSelectModel(config, debug=True)

### (c) Training Data

Training data for our purposes consists of **traces** of generator executions that produce the correct program given an input-output example. These traces store the correct choices made by the operators and hence are suffice as training data for the operators.

In [14]:
import pickle
import random
from urllib.request import urlopen

data = pickle.load(urlopen("https://risecamp2019-atlas.s3.us-east-2.amazonaws.com/training_data_pivot_columns.pkl"))
data[100]


        GeneratorTrace(inputs=((      0           1     2       3                 4   5
0   baz       123.4   baz  Joseph      banana_foo_2  50
1   bar  23894243.7  fizz     Amy       date_bar_24  30
2  buzz       123.4  buzz    Anne  cherimoya_baz_71  20
3   foo       123.4  buzz  Nikita      apple_fizz_7  35,                             1                          2                      \
3                         Amy   Anne Joseph Nikita   Amy  Anne Joseph Nikita   
4                                                                              
apple_fizz_7              NaN    NaN    NaN  123.4   NaN   NaN    NaN   buzz   
banana_foo_2              NaN    NaN  123.4    NaN   NaN   NaN    baz    NaN   
cherimoya_baz_71          NaN  123.4    NaN    NaN   NaN  buzz    NaN    NaN   
date_bar_24       2.38942e+07    NaN    NaN    NaN  fizz   NaN    NaN    NaN   

                    5                     
3                 Amy Anne Joseph Nikita  
4                                      

In [15]:
model = PivotGeneratorModel()
train = data[:500]
valid = data[500:]
model.train(train, valid, num_epochs=30)

100%|██████████| 100/100 [00:00<00:00, 2707.61it/s]


[+] Training model for OpInfo(sid='/gen_pivot_args/Select@select_columns@1', gen_name='gen_pivot_args', op_type='Select', index=1, gen_group=None, uid='select_columns', tags=None)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


/usr/local/lib/python3.7/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


[Training(1/30)] Loss:  1.433152 Accuracy:  0.4720
[Validation(1/30)] Loss:  1.241686 Accuracy:  0.3800
[Training(2/30)] Loss:  1.322016 Accuracy:  0.4580
[Validation(2/30)] Loss:  1.044748 Accuracy:  0.3300
[Training(3/30)] Loss:  1.124238 Accuracy:  0.4480
[Validation(3/30)] Loss:  1.082418 Accuracy:  0.3500
[Training(4/30)] Loss:  1.123267 Accuracy:  0.4680
[Validation(4/30)] Loss:  1.071257 Accuracy:  0.3900
[Training(5/30)] Loss:  1.102100 Accuracy:  0.4720
[Validation(5/30)] Loss:  1.030015 Accuracy:  0.3900
[Training(6/30)] Loss:  1.062197 Accuracy:  0.4760
[Validation(6/30)] Loss:  1.004234 Accuracy:  0.3900
[Training(7/30)] Loss:  1.043536 Accuracy:  0.4800
[Validation(7/30)] Loss:  0.992720 Accuracy:  0.3800
[Training(8/30)] Loss:  1.030523 Accuracy:  0.4980
[Validation(8/30)] Loss:  0.992106 Accuracy:  0.4000
[Training(9/30)] Loss:  1.018177 Accuracy:  0.5020
[Validation(9/30)] Loss:  0.987970 Accuracy:  0.4100
[Training(10/30)] Loss:  1.005545 Accuracy:  0.5040
[Validation(

In [16]:
def synthesize_pivot(inp, out):
    start_time = time.time()
    num_explored = 0
    num_errors = 0
    for args in gen_pivot_args.generate(inp, out).with_model(model):  # CHANGED : Using model
        num_explored += 1
        
        try:
            # If there are exceptions while running pivot or checking, skip
            result = inp.pivot(**args)
            if check_equal(result, out):
                print("Found Solution!", 
                      f"inp.pivot(index='{args['index']}', columns='{args['columns']}', values='{args['values']}')")
                print(f"Time to solution: {time.time() - start_time: .3f} seconds")
                print(f"Number of Programs Explored Till Now: {num_explored}")

        except Exception as e:
            num_errors += 1
            continue
            
    print(f"Total Number of Programs Explored: {num_explored}")
    print(f"Total Number of Programs Crashed: {num_errors}")

In [17]:
synthesize_pivot(inp_df, out_df)

Inference for operator /gen_pivot_args/Select@select_columns@1
[('Category', 0.9999907), ('Expense', 8.901682e-06), ('Date', 2.2617908e-07), ('Added By', 2.7232154e-09), ('Location', 1.2903171e-09), ('Balance', 5.8135863e-10)]
Found Solution! inp.pivot(index='Date', columns='Category', values='Expense')
Time to solution:  1.017 seconds
Number of Programs Explored Till Now: 336
Total Number of Programs Explored: 4056
Total Number of Programs Crashed: 138


Much better! The model for the first `Select` helped it pick the right value in the first try (and with 99% confidence) which significantly improved the search time. Why stop at using a model for only one `Select`. We can use models for all the operators!

### (d) Defining a Model for All Operators

First let's give the same context to all the operators and not just the first `Select`.

In [18]:
@generator
def gen_pivot_args(input_df: pd.DataFrame, output_df: pd.DataFrame):
    # Select one of columns
    arg_columns = Select(list(input_df.columns), context=(input_df, output_df))
    # Select one of columns or None
    arg_index = Select([None] + [i for i in list(input_df.columns) if i != arg_columns], context=(input_df, output_df))

    # Select one of columns or list of columns
    if Select([True, False], context=(input_df, output_df), uid="branch"):
        arg_values = Select([None] + [i for i in list(input_df.columns) if i != arg_columns and i != arg_index],
                            context=(input_df, output_df))
    else:
        arg_values = list(OrderedSubset([i for i in list(input_df.columns) if i != arg_columns and i != arg_index],
                                        context=(input_df, output_df)))

    return {'index': arg_index, 'columns': arg_columns, 'values': arg_values}

Let's define the model as before, covering all the operators this time.

In [19]:
common_config = {
    'learning_rate': 0.01,
    'node_dimension': 50,
    'classifier_hidden_dims': [50],
    'batch_size': 30000,
    'layer_timesteps': [1, 1, 1]
}

class PivotGeneratorModel(IndependentOperatorsModel):
    @operator
    def Select(*args, **kwargs):
        return PivotSelectModel(common_config)

    @operator(name='Select', uid="branch")
    def SelectBranch(*args, **kwargs):
        return PivotClassifyModel(common_config, domain_size=2)

    @operator
    def OrderedSubset(*args, **kwargs):
        return PivotOrderedSubsetModel(common_config)

As before, we can train the model using the following commands.

```python
model = PivotGeneratorModel()
data = pickle.load(urlopen("https://risecamp2019-atlas.s3.us-east-2.amazonaws.com/training_data_pivot_full.pkl"))
train_data = data[:500]
valid_data = data[500:]
model.train(train_data, valid_data, num_epochs=100)
```

Since training for 100 epochs would take some time, we'll load a pre-trained model (obtained through the same set of commands).

In [20]:
from atlas.models.utils import restore_model
model = restore_model("https://risecamp2019-atlas.s3.us-east-2.amazonaws.com/pandas-pivot-model-full.zip", from_url=True)

/usr/local/lib/python3.7/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Restoring parameters from /var/folders/s9/__w2d9dx2ljdx9qk865hh5qs559bh9/T/tmp3_e7yk6c/models/gen_pivot_args/Select@@1/model.weights
INFO:tensorflow:Restoring parameters from /var/folders/s9/__w2d9dx2ljdx9qk865hh5qs559bh9/T/tmp3_e7yk6c/models/gen_pivot_args/Select@@2/model.weights
INFO:tensorflow:Restoring parameters from /var/folders/s9/__w2d9dx2ljdx9qk865hh5qs559bh9/T/tmp3_e7yk6c/models/gen_pivot_args/Select@branch@1/model.weights
INFO:tensorflow:Restoring parameters from /var/folders/s9/__w2d9dx2ljdx9qk865hh5qs559bh9/T/tmp3_e7yk6c/models/gen_pivot_args/Select@@3/model.weights
INFO:tensorflow:Restoring parameters from /var/folders/s9/__w2d9dx2ljdx9qk865hh5qs559bh9/T/tmp3_e7yk6c/models/gen_pivot_args/OrderedSubset@@1/model.weights


In [21]:
synthesize_pivot(inp_df, out_df)

Found Solution! inp.pivot(index='Date', columns='Category', values='Expense')
Time to solution:  0.037 seconds
Number of Programs Explored Till Now: 1
Total Number of Programs Explored: 2736
Total Number of Programs Crashed: 140


In [22]:
inp_df = pd.DataFrame({
  'foo': ['one', 'one', 'one', 'two', 'two', 'two'],
  'bar': ['A', 'B', 'C', 'A', 'B', 'C'],
  'baz': [10, 20, 30, 40, 50, 60],
})

out_df = pd.DataFrame({
    'A': {'one': 10, 'two': 40},
    'B': {'one': 20, 'two': 50},
    'C': {'one': 30, 'two': 60}
})

synthesize_pivot(inp_df, out_df)

Found Solution! inp.pivot(index='foo', columns='bar', values='baz')
Time to solution:  0.034 seconds
Number of Programs Explored Till Now: 1
Total Number of Programs Explored: 48
Total Number of Programs Crashed: 0


Our synthesizer gets it right on the first try on both occasions.

## Conclusion

In this tutorial, we've learnt to express a search space using generators in Atlas. A Generator is a regular Python function which uses some special operators to capture non-deterministic decisions (such as selecting one element from a list). We built a generator for the `pivot` function in Pandas which helped us construct a synthesizer, which works fine for small inputs but suffers on large ones as the search space is too large. We then used graph-based neural-network models to *guide* the generator towards programs in the search space that are most likely to produce the output given the input. In fact, our learned synthesizer gets the correct program on the first try.